In [29]:
%pwd

'/home/mw/project'

time: 1.72 ms


In [4]:
%load_ext klab-autotime

In [14]:
import kabuki
import hddm
import matplotlib.pyplot as plt
import pickle
from functools import partial
from p_tqdm import p_map
from glob import glob

time: 580 µs


In [6]:
data = hddm.load_csv('/home/mw/input/cpp7572/data.csv')
data.rename(columns={'response_time':'rt','response_corr':'response','subject':'subj_idx'},inplace = True)
data['subj_idx']=data['subj_idx'].str.replace(r'sub-0',r'0')
data = data.drop(data[data['cpp_peak']==100].index)
data

,rt,response,subj_idx,attention,coherency,prioritization,category,cpp_peak,cpp_slope,cpp_amplitude
0,0.421184,1,017,right,low,yes,face,1.337846e-06,0.000010,1.785131e-06
1,0.327997,1,017,double,high,no,face,1.104632e-06,0.000047,7.713533e-08
2,0.230964,1,017,right,high,yes,face,4.495879e-06,0.000007,3.170688e-06
3,0.284764,1,017,double,low,no,car,1.956533e-06,-0.000022,-2.168577e-06
4,0.516040,1,017,double,low,no,car,1.842128e-06,-0.000060,-3.771772e-06
5,0.377975,1,017,left,low,yes,car,7.198718e-07,0.000021,-1.114468e-06
6,0.388672,0,017,double,low,no,face,4.328616e-06,-0.000001,6.473992e-06
7,0.311690,1,017,right,high,yes,car,4.530776e-06,0.000019,-4.726571e-07
8,0.383053,0,017,double,low,no,face,1.964945e-05,-0.000081,5.106468e-06
9,0.251344,1,017,left,high,yes,face,5.402271e-06,0.000065,7.108547e-06


time: 45.8 ms


In [7]:
def run_m2(id, df=None, samples=None, burn=None, save_name=None): 
#     print('running model %i'%id);
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id    
    v_reg = {'model':'v~1+C(coherency, Treatment("low"))+cpp_slope+cpp_slope:C(coherency, Treatment("low"))','link_func':lambda x:x}
    t_reg = {'model':'t~1+C(prioritization, Treatment("no"))','link_func':lambda x:x}
    m = hddm.HDDMRegressor(data, [v_reg, t_reg],include='st', group_only_regressors = False)
    m.find_starting_values()
    m.sample(samples, burn=burn, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

time: 1.14 ms


In [9]:
models_2 = p_map(partial(run_m2, 
                         df = data, 
                         samples = 500,
                         burn = 100,
                         save_name = '/home/mw/project/m2'),
                 range(8))

Adding these covariates:
['v_Intercept', 'v_C(coherency, Treatment("low"))[T.high]', 'v_cpp_slope', 'v_cpp_slope:C(coherency, Treatment("low"))[T.high]']
Adding these covariates:
['v_Intercept', 'v_C(coherency, Treatment("low"))[T.high]', 'v_cpp_slope', 'v_cpp_slope:C(coherency, Treatment("low"))[T.high]']
['t_Intercept', 't_C(prioritization, Treatment("no"))[T.yes]']
Adding these covariates:
Adding these covariates:
['t_Intercept', 't_C(prioritization, Treatment("no"))[T.yes]']
Adding these covariates:
['v_Intercept', 'v_C(coherency, Treatment("low"))[T.high]', 'v_cpp_slope', 'v_cpp_slope:C(coherency, Treatment("low"))[T.high]']
Adding these covariates:
['t_Intercept', 't_C(prioritization, Treatment("no"))[T.yes]']
Adding these covariates:
['v_Intercept', 'v_C(coherency, Treatment("low"))[T.high]', 'v_cpp_slope', 'v_cpp_slope:C(coherency, Treatment("low"))[T.high]']
Adding these covariates:
['t_Intercept', 't_C(prioritization, Treatment("no"))[T.yes]']
Adding these covariates:
['v_Int

/opt/conda/lib/python3.6/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.6/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.6/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.6/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.6/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.6/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.6/site-pack

 [-----------------96%----------------  ] 480 of 500 complete in 1189.3 secWARNING: Will not save custom link functions.
 [-----------------100%-----------------] 501 of 500 complete in 1265.7 secWARNING: Will not save custom link functions.
 [-----------------100%-----------------] 501 of 500 complete in 1987.5 secWARNING: Will not save custom link functions.
 [-----------------100%-----------------] 501 of 500 complete in 2930.3 secWARNING: Will not save custom link functions.



AssertionError: Step-out procedure failed

time: 49min 9s
 [-----------------92%--------------    ] 460 of 500 complete in 5243.9 sec

In [19]:
models_2_comb = []
for model_path in glob('/home/mw/project/m2_chain_*'):
    if ('db' not in model_path) and ('csv' not in model_path):
        print(model_path)
        model2 = hddm.load(model_path)
        models_2_comb.append(model2)

/home/mw/project/m2_chain_0
/home/mw/project/m2_chain_2
/home/mw/project/m2_chain_1
/home/mw/project/m2_chain_5
time: 10.1 s


In [20]:
m2 = kabuki.utils.concat_models(models_2_comb)

time: 2.68 s


In [21]:
m2.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
a,1.12471,0.0869502,0.971473,1.06832,1.11842,1.17515,1.31746,0.00298516
a_std,0.252347,0.082665,0.144926,0.196737,0.234789,0.288569,0.471192,0.00337116
a_subj.001,1.13315,0.0737501,1.00946,1.08336,1.12759,1.1757,1.2936,0.0035435
a_subj.003,1.20561,0.0580287,1.10142,1.16528,1.20291,1.24365,1.32741,0.00267785
a_subj.005,1.02198,0.0622425,0.909087,0.98024,1.01661,1.06476,1.14592,0.00293685
a_subj.006,1.48213,0.0869826,1.32848,1.41976,1.47784,1.53888,1.66778,0.00442229
a_subj.008,1.08323,0.0717348,0.956196,1.03329,1.07872,1.12859,1.2371,0.00362108
a_subj.010,0.729886,0.045752,0.643946,0.699073,0.727787,0.759443,0.826128,0.00236159
a_subj.012,0.993624,0.0571458,0.882271,0.956408,0.992774,1.0303,1.11526,0.00264758
a_subj.015,0.990929,0.052129,0.899573,0.952005,0.987681,1.02507,1.09515,0.00235142


time: 218 ms


In [27]:
m2.get_traces().to_csv('/home/mw/temp/m2_trace.csv',index=False)

time: 288 ms


In [22]:
v_cpp,v_coh,v_cpp_coh = m2.nodes_db.loc[['v_cpp_slope',
                                'v_C(coherency, Treatment("low"))[T.high]',
                                'v_cpp_slope:C(coherency, Treatment("low"))[T.high]'],'node']
hddm.analyze.plot_posterior_nodes([v_cpp,v_coh,v_cpp_coh])
plt.xlabel('Regression coefficient for cpp')
plt.ylabel('Posterior probability')

Text(0, 0.5, 'Posterior probability')

<Figure size 432x288 with 1 Axes>

time: 205 ms


In [28]:
m2.dic

-2165.5554200605225

time: 35.7 ms
